# 🧠 AI Agents Course – Section 3
## 📌 Notebook: AI Workflow - Email writing

---

### 🚀 Objective:
Compare between writing an email with a simple prompt, and with a simple AI workflow with roleplay. 

--- 
###   Tasks: 
After trying this, I want you to experiment with a Marketing Agent that uses a simple AI workflow to write an SEO enhanced Blog Post.

---

_Hassan Tabbal • Introduction to AI Agents Development Training — All rights reserved_

In [1]:
from pydantic_ai import Agent,RunContext
from textwrap import dedent # allows us to format string nicely in code, but remove unwanted white space before sending to LLM
from IPython.display import Markdown,display_pretty
import nest_asyncio # needed to fix an error when running inside Jupyter notebook
nest_asyncio.apply()

model = "google-gla:gemini-2.0-flash"
agent = Agent(  
    model,
    system_prompt='',  
)

In [ ]:
system_prompt_simple = dedent("""
You are a professional public relations expert who specializes in writing perfect emails. When writing an email, make sure that the goal is clearly defined and that you have all the info required to craft the email. 
                              Avoid jargon or cliche sentences that are easily detectable as AI generated.
                              Each email shall have a clear purpose, defining why it is sent and what is expected from the recipient.

""")
agent = Agent(  
    model,
    system_prompt=system_prompt_simple,
    result_retries=3  
)
result = agent.run_sync(dedent("""
                               I am a freshly graduated Computer Engineering student from Lebanese International University (LIU).
                               I want to apply to the position of 'Full Stack Developer' at a company called 'NavyBits'.
                               If they don't have an open position, I want them to consider me for an internship.
                               Follow a professional and persuasive tone.
                                """))  
Markdown(result.output)

Now let's enhance the output with a simple workflow

In [ ]:
import time
from pydantic import BaseModel
class EmailModel(BaseModel):
    emailSubject:str # The subject of the email
    emailBody:str # The body of the email

class EmailEvaluation(BaseModel):
    emailScore:int # A score of the likelihood of the email to be successful
    enhancementsInstructions:str # instructions on how to enhance the email    

system_prompt_simple = dedent("""
You are a professional public relations expert who specializes in writing perfect emails. When writing an email, make sure that the goal is clearly defined and that you have all the info required to craft the email. 
                              Avoid jargon or cliche sentences that are easily detectable as AI generated.
                              Each email shall have a clear purpose, defining why it is sent and what is expected from the recipient.

""")
agent = Agent(  
    model,
    system_prompt=system_prompt_simple,
)
MAX_LOOPS = 6
current_loop = 1
result = agent.run_sync(dedent("""
                               I am a freshly graduated Computer Engineering student from Lebanese International University (LIU).
                               I want to apply for an internship as an  'AI Agents Developer' at a company called 'NavyBits'.
                               Follow a professional and persuasive tone.
                                """),output_type=EmailModel)  
evaluation = agent.run_sync(dedent(f""""
                                       Assume the role of the recipient of the email. 
                                       Imagine that the recipient is a C-level business executive who is bombarded daily by multiple emails.
                                       Analyze the goal of the email, then provide a score on a scale from 1 to 10, 1 being very bad and 10 being excellent, on the likelihood of being convinced by the email.
                                       Also provide suggestions for further enhancements. 
                                       #Email: 
                                        *subject* : {result.output.emailSubject}
                                        *body*: {result.output.emailBody}
                                    
"""),output_type=EmailEvaluation)
print(f"BEFORE - Score = {evaluation.output.emailScore}")
print(f"Subject : {result.output.emailSubject}")
print(result.output.emailBody)
#print(result.output.model_dump_json(indent=4))
while current_loop <= MAX_LOOPS and evaluation.output.emailScore < 8: 
    result = agent.run_sync(dedent(f"""
                              Apply the suggested enhancements to enhance the email.
                              #Initial Email : 
                                *subject* : {result.output.emailSubject}
                                *body* : {result.output.emailBody}
                              #Enhancements : {evaluation.output.enhancementsInstructions}
                                """),output_type=EmailModel) 
    evaluation = agent.run_sync(dedent(f""""
                                       Assume the role of the recipient of the email. 
                                       Imagine that the recipient is a C-level business executive who is bombarded daily by multiple emails.
                                       Analyze the goal of the email, then provide a score on a scale from 1 to 10, 1 being very bad and 10 being excellent, on the likelihood of being convinced by the email.
                                       Also provide suggestions for further enhancements. 
                                       #Email: 
                                       *subject* : {result.output.emailSubject}
                                       *body* : {result.output.emailBody}
    """),output_type=EmailEvaluation)
    #print(evaluation.output.model_dump_json(indent=4))

    current_loop += 1
print(f"AFTER - Score = {evaluation.output.emailScore}")
print(f"Subject : {result.output.emailSubject}")
print(result.output.emailBody)

BUT do we really need to pass the message history one by one? Why are we using a framework if we have to maintain the history manually?

In [3]:
import time
from pydantic import BaseModel
class EmailModel(BaseModel):
    emailSubject:str # The subject of the email
    emailBody:str # The body of the email

class EmailEvaluation(BaseModel):
    emailScore:int # A score of the likelihood of the email to be successful
    enhancementsInstructions:str # instructions on how to enhance the email    

system_prompt_simple = dedent("""
You are a professional public relations expert who specializes in writing perfect emails. When writing an email, make sure that the goal is clearly defined and that you have all the info required to craft the email. 
                              Avoid jargon or cliche sentences that are easily detectable as AI generated.
                              Each email shall have a clear purpose, defining why it is sent and what is expected from the recipient.

""")
agent = Agent(  
    model,
    system_prompt=system_prompt_simple,
)
MAX_LOOPS = 6
current_loop = 1
result = agent.run_sync(dedent("""
                               I am a freshly graduated Computer Engineering student from Lebanese International University (LIU).
                               I want to apply for an internship as an  'AI Agents Developer' at a company called 'NavyBits'.
                               Follow a professional and persuasive tone.
                                """),output_type=EmailModel)  
evaluation = agent.run_sync(dedent(f""""
                                       Assume the role of the recipient of the email. 
                                       Imagine that the recipient is a C-level business executive who is bombarded daily by multiple emails.
                                       Analyze the goal of the email, then provide a score on a scale from 1 to 10, 1 being very bad and 10 being excellent, on the likelihood of being convinced by the email.
                                       Also provide suggestions for further enhancements. 
                                    
"""),output_type=EmailEvaluation,message_history=result.all_messages())
print(f"BEFORE - Score = {evaluation.output.emailScore}")
print(f"Subject : {result.output.emailSubject}")
print(result.output.emailBody)
#print(result.output.model_dump_json(indent=4))
while current_loop <= MAX_LOOPS and evaluation.output.emailScore < 8: 
    result = agent.run_sync(dedent(f"""
                              Apply the suggested enhancements to enhance the email.
                             """),output_type=EmailModel,message_history=evaluation.all_messages()) 
    evaluation = agent.run_sync(dedent(f""""
                                       Assume the role of the recipient of the email. 
                                       Imagine that the recipient is a C-level business executive who is bombarded daily by multiple emails.
                                       You hate emails generated by AI.
                                       Ignore placeholders and focus only on the email text without placeholders.
                                       Analyze the goal of the email, then provide a score on a scale from 1 to 10, 1 being very bad and 10 being excellent, on the likelihood of being convinced by the email.
                                       Always give a low score if you suspect the email is generated by AI.
                                       Also provide suggestions for further enhancements. 
    """),output_type=EmailEvaluation,message_history=result.all_messages())
    #print(evaluation.output.model_dump_json(indent=4))

    current_loop += 1
print(f"AFTER - Score = {evaluation.output.emailScore}")
print(f"Subject : {result.output.emailSubject}")
print(result.output.emailBody)

BEFORE - Score = 7
Subject : AI Agents Developer Internship Application
Dear NavyBits Hiring Team,

I am writing to express my keen interest in the AI Agents Developer internship opportunity at NavyBits. As a recent Computer Engineering graduate from Lebanese International University (LIU), I have developed a strong foundation in AI principles and a passion for creating intelligent systems.

During my studies at LIU, I gained hands-on experience in developing and implementing AI algorithms, with a particular focus on agent-based modeling and reinforcement learning. I am proficient in Python and have experience with relevant libraries such as TensorFlow and PyTorch. My academic projects include [mention specific projects relevant to AI agents, if any].

I am eager to apply my skills and contribute to NavyBits' innovative projects. I am a quick learner, a dedicated team player, and I am confident that I can make a valuable contribution to your team.

Thank you for considering my applicat